## Cloud identification 

Importando as bibliotecas necessárias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import warnings
from scipy.optimize import OptimizeWarning
from scipy.optimize import curve_fit
import os
import math
import cv2

Cálculo para identificar os pixels caracterizados como nuvens (condição de céu encoberto)

In [ ]:
nubladas = '/home/personal/figures'
cards =[]
for imagem in nubladas:
    limite_nuvem = 1.25
    white = 255
    black = 0

    im = cv2.imread('Imagem/1.25/' + imagem)

    red = [0,0,2]
    blue = [0,0,1]
    green = [0,0,0]


    (red,blue,green) = cv2.split(im)
    print((red,blue,green))

    rbr = (red/blue)
    print('O RBR é: ',rbr)
    razao = (rbr/limite_nuvem)
    print('a razão é', razao)
    max_rbr = np.amax(rbr)
    print('O valor máximo do RBR é: ', max_rbr)
    min_rbr = np.amin(rbr)
    print('O valor mínimo do RBR é: ', min_rbr)

    local_nuvem = np.where(rbr<= limite_nuvem)
    print(local_nuvem)

    tamanho = np.shape(local_nuvem)
    print(tamanho)
            
    pontos_x = local_nuvem[0]
    pontos_y = local_nuvem[1]

    ### Parâmetros para construir a imagem
    rows = 1
    cols = 2
    axes=[]
    fig, (ax1, ax2) = plt.subplots(nrows=rows, ncols=cols,figsize=(30,15))
    #fig.add_subplot(rows, cols, 1)
    ax1.axis("off")
    ax1.title.set_text('Original')
    #fig.add_subplot(rows, cols, 2)
    ax2.axis("off")
    ax2.title.set_text("Limiar nuvem/não-nuvem")

    ax1.imshow(im[:,:,[2,1,0]])
    ax2.scatter(x=pontos_y,y=pontos_x,c='red')
    ax2.imshow(im[:,:,[2,1,0]])

    plt.show()


    card={}
    card['Nome da Imagem'] = imagem
    card['Valor máximo do RBR'] = max_rbr
    card['Valor mínimo do RBR'] = min_rbr
    card['Pontos onde há nebulosidade'] = local_nuvem
    cards.append(card)

O polinômio da biblioteca de céu claro

In [2]:
ceu_claro = pd.read_csv('Planilhas_rbr/Planilha_unida/cos_gama_razao_rbr_filtrado.csv')
teste = pd.read_csv('informacoes_pixels_imagem.csv')
polinomio_cosseno = pd.read_csv('cosseno_gama_do_polinomio.csv')
polinomio_zenite = pd.read_csv('informacoes_imagem_polinomio.csv')
rbr = pd.read_csv('Planilhas_rbr/20200318150000_11.jpg_rbr_pixels_polinomio.csv')

In [6]:
# Supondo que df seja o seu DataFrame

def sua_funcao(x, *params):
    i, j = x
    return sum(params[k] * x[0]**i * x[1]**j for k, (i, j) in enumerate([(i, j) for i in range(5) for j in range(5)]))

# Ajuste da função polinomial aos dados
initial_guess = [1.0] * 25  # valores iniciais para os 25 coeficientes (5x5)
#initial_guess
# Ajuste da função polinomial aos dados
params, covariance = curve_fit(sua_funcao, (ceu_claro['zenite_pixel'], ceu_claro['cosseno_gama']), ceu_claro['limiar_rbr'], p0=initial_guess)

# Imprime os parâmetros ajustados
# print("Parâmetros ajustados:", params)
# Extrair a matriz de covariância
covariance_matrix = np.sqrt(np.diag(covariance))

# Imprimir os parâmetros ajustados e as incertezas associadas
for i, (param, uncertainty) in enumerate(zip(params, covariance_matrix)):
    print(f"Parâmetro {i}: {param}, Incerteza: {uncertainty}")

Parâmetro 0: 1.8659677833134984, Incerteza: 0.06694679687902648
Parâmetro 1: -0.4516800269483274, Incerteza: 0.18228322672337555
Parâmetro 2: -1.045109083743662, Incerteza: 0.17015149195121915
Parâmetro 3: 0.8684120217871955, Incerteza: 0.06526071301118992
Parâmetro 4: -0.16674407983696068, Incerteza: 0.008814506542679637
Parâmetro 5: -7.3487797676777955, Incerteza: 0.4978681760912158
Parâmetro 6: 8.124292893633752, Incerteza: 1.3367925897916968
Parâmetro 7: 2.32295833702298, Incerteza: 1.238352142118277
Parâmetro 8: -3.95127722308725, Incerteza: 0.47343806257689885
Parâmetro 9: 0.849380507458687, Incerteza: 0.06389118310158992
Parâmetro 10: 19.12438055307753, Incerteza: 1.2161929167693453
Parâmetro 11: -25.6135063347063, Incerteza: 3.226765883240445
Parâmetro 12: 3.4088594607406506, Incerteza: 2.962296395211441
Parâmetro 13: 4.767605054525422, Incerteza: 1.1264801406480986
Parâmetro 14: -1.2484959577663244, Incerteza: 0.15162861633967964
Parâmetro 15: -15.159836068301045, Incerteza: 1

In [8]:
valor_polinomio = sua_funcao((ceu_claro['zenite_pixel'], ceu_claro['cosseno_gama']), *params)
valor_polinomio

0         1.119937
1         1.118728
2         1.115728
3         1.112398
4         1.109313
            ...   
493855    1.215026
493856    1.215060
493857    1.215138
493858    1.215255
493859    1.215404
Length: 493860, dtype: float64

In [ ]:

#### Calculando a função do threshold adaptativo ####
figura1 = cv2.imread('figuras/20200317150000_11.jpg')
polinomio_imagem1 = pd.read_csv('processadas/informacoes_imagem_20200317150000_processado.csv')
# Encontrar índices dos valores NaN
print(polinomio_imagem1.isna().any())
polinomio_imagem1['cosseno angulo'] = polinomio_imagem1['cosseno angulo'].fillna(0)
print(polinomio_imagem1)
figura1= cv2.flip(figura1, 1)
# Split channels
# Separar os canais de cores
blue, green, red = cv2.split(figura1)

# Replace zeros in blue channel with a small positive value to avoid division by zero
blue[blue == 0] = 1

# Calculate RBR
rbr = red / blue
#print(rbr)
### Cálculoda função 
valor_1 = sua_funcao((polinomio_imagem1['zenite pixel'],polinomio_imagem1['cosseno angulo']),*params)
print(valor_1)
# Redimensionar valor_1 para ter as mesmas dimensões que rbr
valor_1_resized = np.resize(valor_1, rbr.shape)

# Calcular diferenças entre rbr e valor_1
diferencas = rbr - valor_1_resized
print(diferencas)

# Redimensionar polinomio_imagem1['cosseno angulo'] para que tenha o mesmo formato que diferencas
polinomio_reshaped = np.resize(polinomio_imagem1['cosseno angulo'],rbr.shape)

# Calcular diff aplicando a função
diff = diferencas * (1 + np.exp(-(np.abs(polinomio_reshaped)))/5)
print(diff.min())


# Plotar a imagem das diferenças em cores RGB
plt.imshow(diff, cmap='binary')  # Usar cmap='gray' para representar como escala de cinza
plt.title('Imagem das diferenças (RGB)')
plt.axis('off')
plt.colorbar()
plt.show()

